In [1]:
import pandas as pd
import numpy as np
import optuna
from sklearn.svm import SVR
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt

In [2]:
train = pd.read_csv(r"C:\Users\YILMAZ\Desktop\train_fe_scaled.csv")
test = pd.read_csv(r"C:\Users\YILMAZ\Desktop\test_fe_scaled.csv")

target = "price_z"
X_train = train.drop(columns=[target])
y_train = train[target]

In [3]:
def objective(trial):
    params = {
        'C': trial.suggest_float('C', 0.1, 100.0, log=True),
        'epsilon': trial.suggest_float('epsilon', 1e-3, 1.0, log=True),
        'kernel': trial.suggest_categorical('kernel', ['linear', 'rbf'])
    }

    model = SVR(**params)

    cv = KFold(n_splits=5, shuffle=True, random_state=42)
    rmse = -cross_val_score(model, X_train, y_train, 
                            scoring='neg_root_mean_squared_error', 
                            cv=cv, 
                            n_jobs=-1).mean()
    return rmse

In [4]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=30, timeout=600)

[I 2025-05-21 23:50:06,531] A new study created in memory with name: no-name-2e4ae7d1-9798-4293-b262-09ab4dd1e3b1
[I 2025-05-22 00:43:17,440] Trial 0 finished with value: 226843.8560323398 and parameters: {'C': 4.260093837213218, 'epsilon': 0.08672241681866034, 'kernel': 'linear'}. Best is trial 0 with value: 226843.8560323398.


In [5]:
print("\n Best hyperparameters (SVR):")
print(study.best_params)


 Best hyperparameters (SVR):
{'C': 4.260093837213218, 'epsilon': 0.08672241681866034, 'kernel': 'linear'}


In [6]:
best_model = SVR(**study.best_params)
best_model.fit(X_train, y_train)

SVR(C=4.260093837213218, epsilon=0.08672241681866034, kernel='linear')

In [7]:
cv = KFold(n_splits=5, shuffle=True, random_state=42)
cv_rmse = -cross_val_score(best_model, X_train, y_train, scoring='neg_root_mean_squared_error', cv=cv)
cv_mae = -cross_val_score(best_model, X_train, y_train, scoring='neg_mean_absolute_error', cv=cv)
cv_r2 = cross_val_score(best_model, X_train, y_train, scoring='r2', cv=cv)

print("\n Tuned SVR 5-Fold Cross-Validation Results: ")
print(f"🔹 RMSE: {cv_rmse.mean():,.2f} ± {cv_rmse.std():,.2f}")
print(f"🔹 MAE: {cv_mae.mean():,.2f} ± {cv_mae.std():,.2f}")
print(f"🔹 R²: {cv_r2.mean():.4f} ± {cv_r2.std():.4f}")


 Tuned SVR 5-Fold Cross-Validation Results:
🔹 RMSE: 226,843.86 ± 4,347.37
🔹 MAE: 126,795.52 ± 1,136.36
🔹 R²: 0.7219 ± 0.0049
